In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [4]:
ratings = pd.read_csv('ml-latest/ratings.csv')
print(ratings.head())
  
movies = pd.read_csv("movies_FINAL.csv")
print(movies.head())
  
n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

   userId  movieId  rating   timestamp
0       1      307     3.5  1256677221
1       1      481     3.5  1256677456
2       1     1091     1.5  1256677471
3       1     1257     4.5  1256677460
4       1     1449     4.5  1256677264
   movieId                        title  year    rating  Western  Film-Noir  \
0        1                    Toy Story  1995  3.886649        0          0   
1        2                      Jumanji  1995  3.246583        0          0   
2        3             Grumpier Old Men  1995  3.173981        0          0   
3        4            Waiting to Exhale  1995  2.874540        0          0   
4        5  Father of the Bride Part II  1995  3.077291        0          0   

   IMAX  Musical  Sci-Fi  Adventure  ...  Action  War  Mystery  Animation  \
0     0        0       0          1  ...       0    0        0          1   
1     0        0       0          1  ...       0    0        0          0   
2     0        0       0          0  ...       0    0       

In [5]:
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")

Number of ratings: 27753444
Number of unique movieId's: 53889
Number of unique users: 283228
Average ratings per user: 97.99
Average ratings per movie: 515.01


In [6]:
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
user_freq.head()

userId  n_ratings
0       1         16
1       2         15
2       3         11
3       4        736
4       5         72

In [7]:
mean_rating = ratings.groupby('movieId')[['rating']].mean()
# Lowest rated movies
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]
# Highest rated movies
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]
# show number of people who rated movies rated movie highest
ratings[ratings['movieId']==highest_rated]
# show number of people who rated movies rated movie lowest
ratings[ratings['movieId']==lowest_rated]
  
## the above movies has very low dataset. We will use bayesian average
movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

In [8]:
from scipy.sparse import csr_matrix
  
def create_matrix(df):
      
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
      
    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
      
    # Map indices to IDs
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
      
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
  
    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
      
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [9]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

In [26]:


def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
      
    neighbour_ids = []
      
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [33]:
movie_titles = dict(zip(movies['movieId'], movies['title']))

In [59]:
movie_id = 112
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]
print(f"Since you watched {movie_title}\n")
for i in similar_ids:
    print(movie_titles[i])

Since you watched Rumble in the Bronx (Hont faan kui)

Broken Arrow
Eraser
Rock, The
Executive Decision
Mission: Impossible
Twister
Independence Day (a.k.a. ID4)
Heat
Supercop (Police Story 3: Supercop) (Jing cha gu shi III: Chao ji jing cha)
First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo)
